In [ ]:
import os
import time
import datetime
import warnings

import tqdm

import IPython

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
tf.__version__

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
tf.keras.backend.clear_session()

def convolve_block(filters, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2D(
            filters, 3, strides=1, padding='same',
            kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result


def Model():
    network_stack = [
        convolve_block(64, apply_batchnorm=False),
        convolve_block(128),
        convolve_block(256),
        convolve_block(512),
        convolve_block(512),
        convolve_block(512),
        convolve_block(512),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(8),
    ]

    inputs = tf.keras.layers.Input(shape=[None,None,1], batch_size=None)
    x = inputs

    for block in network_stack:
        x = block(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


model = Model()

tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [ ]:
# model.trainable_variables

In [ ]:
from pymedphys._mocks import wlutz, profiles
from pymedphys._wlutz import reporting

In [ ]:
def create_single_dataset(grid_size):

    while True:
        field_centre = tf.random.uniform((2,), -0.5, 0.5)
        field_side_lengths = tf.random.uniform((2,), 0.2, 1.5)
        field_penumbra = tf.random.uniform((), 0.05, 0.2).numpy()
        field_rotation_raw = tf.random.uniform((), -1, 1).numpy()
        field_rotation = field_rotation_raw * 180

        bb_centre = tf.random.uniform((2,), -0.5, 0.5)
        bb_diameter = tf.random.uniform((), 0.05, 0.3).numpy()
        bb_max_attenuation = tf.random.uniform((), 0.1, 0.5).numpy()


        field = profiles.create_rectangular_field_function(
            field_centre, field_side_lengths, field_penumbra, field_rotation
        )
        bb_penumbra = field_penumbra / 3
        bb_attenuation_map = wlutz.create_bb_attenuation_func(
            bb_diameter, bb_penumbra, bb_max_attenuation
        )

        x = np.linspace(-1, 1, grid_size)
        xx, yy = np.meshgrid(x, x)

        without_bb = field(xx, yy)

        def field_with_bb(x, y):
            return field(x, y) * bb_attenuation_map(x - bb_centre[0], y - bb_centre[1])

        with_bb = field_with_bb(xx, yy)
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            log_mean_sqr_diff = np.log(np.mean((without_bb - with_bb)**2))
#         print(log_mean_sqr_diff)

        if log_mean_sqr_diff > -10:
            break
            
    parameters = tf.concat([field_centre, field_side_lengths, [field_rotation_raw], bb_centre, [bb_diameter]], 0)
    img = tf.convert_to_tensor(with_bb, dtype=tf.float32)
            
    return parameters, img

parameters, img = create_single_dataset(128)


plt.figure()
plt.pcolormesh(img)
plt.axis('equal')

In [ ]:
img.shape[0]

In [ ]:
def create_pipeline_dataset(batch_size):
    def dataset_generator():
        image_size_for_current_batch = tf.random.uniform((), 32, 128, dtype=tf.int32).numpy()
        for _ in range(batch_size):
            yield create_single_dataset(image_size_for_current_batch)

    dataset = tf.data.Dataset.from_generator(
        dataset_generator, 
        (tf.float32, tf.float32), 
        (tf.TensorShape([8]), tf.TensorShape([None, None]))
    )

    dataset = dataset.repeat().batch(batch_size)
    
    return dataset

In [ ]:
for parameters, img in create_pipeline_dataset(2).take(3):    
    dim = img.shape
    print(dim)
    for i in range(dim[0]):        
        plt.figure()
        plt.pcolormesh(img[i, :, :])
        plt.axis('equal')
    plt.show()

In [ ]:
def extract_parameters(tensor_parameters):
    parameters = {
        'field_centre': (tensor_parameters[0], tensor_parameters[1]),
        'field_side_lengths': (tensor_parameters[2], tensor_parameters[3]),
        'field_rotation': tensor_parameters[4] * 180,
        'bb_centre': (tensor_parameters[5], tensor_parameters[6]),
        'bb_diameter': tensor_parameters[7]
    }
    
    return parameters

In [ ]:
def create_figure(image, field_centre, field_side_lengths, field_rotation, bb_centre, bb_diameter):
    dim = image.shape
    x = np.linspace(-1, 1, dim[0])
    y = x
    
    return reporting.image_analysis_figure(
        x, y, np.array(image),
        np.array(bb_centre), np.array(field_centre), np.array(field_rotation),
        bb_diameter, field_side_lengths, penumbra=0.2, units=''
    )

In [ ]:
def results_figures(model, batch_images, batch_ground_truth_parameters):
    batch_dim = batch_images.shape
    num_batches = batch_dim[0]
    image_shape = dim[1]
    x = np.linspace(-1, 1, image_shape)
    y = x
    
    batch_predicted_parameters = model(batch_images, training=True)
    
    for i in range(num_batches):
        
        ground_truth_parameters = extract_parameters(batch_ground_truth_parameters[i, :])
        predicted_parameters = extract_parameters(batch_predicted_parameters[i, :])
    
        fig, axs = create_figure(batch_images[i,:,:], **ground_truth_parameters)
        axs[0,0].set_title("Ground Truth")

        fig, axs = create_figure(batch_images[i,:,:], **predicted_parameters)
        axs[0,0].set_title("Predicted")

        plt.show()

In [ ]:
for parameters, img in test_dataset.take(1):
    results_figures(model, img, parameters)

In [ ]:
# def extract_parameters(tensor_parameters):
#     parameters = {
#         'field_centre': (tensor_parameters[0], tensor_parameters[1]),
#         'field_side_lengths': (tensor_parameters[2], tensor_parameters[3]),
#         'field_rotation': tensor_parameters[4] * 180,
#         'bb_centre': (tensor_parameters[5], tensor_parameters[6]),
#         'bb_diameter': tensor_parameters[7]
#     }
    
#     return parameters

In [ ]:
def flip_parameters(parameters):
    new_parameters = tf.concat([
        parameters[:, 0:2],
        parameters[:, 3:4],
        parameters[:, 2:3],
        parameters[:, 4:5] + 0.5,
        parameters[:, 5::]
    ], axis=-1)
    
    return new_parameters


for parameters, img in create_pipeline_dataset(1).take(1):
    flipped_parameters = flip_parameters(parameters)
    
    print(parameters)
    print(flipped_parameters)
    
    batch_dim = img.shape
    num_batches = batch_dim[0]
    
    for i in range(num_batches):
        fig, axs = create_figure(img[i,:,:], **extract_parameters(parameters[i,:]))
        axs[0,0].set_title("Original")

        fig, axs = create_figure(img[i,:,:], **extract_parameters(flipped_parameters[i,:]))
        axs[0,0].set_title("Flipped")

In [ ]:
def determine_rotation_diff(predicted_parameters, ground_truth_parameters):
    predicted_rotations = predicted_parameters[:, 4] * 180
    ground_truth_rotations = ground_truth_parameters[:, 4] * 180
    
    diff_rotation = (predicted_rotations - ground_truth_rotations) % 180
    diff_rotation = tf.reduce_min(tf.stack([diff_rotation, 180 - diff_rotation]), axis=0)
    
    return diff_rotation


def adjust_rotation(parameters, degrees):
    new_parameters = tf.concat([
        parameters[:,0:4],
        parameters[:,4:5] + degrees/180,
        parameters[:,5::]
    ], axis=-1)
    
    return new_parameters


for parameters, img in create_pipeline_dataset(4).take(1):    
    print(determine_rotation_diff(parameters, parameters))
    print(determine_rotation_diff(parameters, adjust_rotation(parameters, 1)))
    print(determine_rotation_diff(parameters, adjust_rotation(parameters, 90)))
    print(determine_rotation_diff(parameters, adjust_rotation(parameters, 180)))
    print(determine_rotation_diff(parameters, adjust_rotation(parameters, 181)))
    print(determine_rotation_diff(parameters, adjust_rotation(parameters, 179)))

In [ ]:
def determine_diff_with_rotation(predicted_parameters, ground_truth_parameters):
    diff = tf.concat([
        predicted_parameters[:, 0:4] - ground_truth_parameters[:, 0:4],
        determine_rotation_diff(predicted_parameters, ground_truth_parameters)[:, None] / 180,
        predicted_parameters[:, 5::] - ground_truth_parameters[:, 5::],
    ], axis=-1)
    
    return diff


for parameters, img in create_pipeline_dataset(4).take(1):
    
    flipped_parameters = flip_parameters(parameters)
    print(determine_diff_with_rotation(parameters, parameters))
    print(determine_diff_with_rotation(parameters, flipped_parameters))
    
    print()
    
    print(determine_diff_with_rotation(parameters, parameters + 1))
    print(determine_diff_with_rotation(parameters, adjust_rotation(parameters, 1)))
    print(determine_diff_with_rotation(parameters, adjust_rotation(parameters, 90)))
    print(determine_diff_with_rotation(parameters, adjust_rotation(parameters, 180)))
    print(determine_diff_with_rotation(parameters, adjust_rotation(parameters, 181)))
    print(determine_diff_with_rotation(parameters, adjust_rotation(parameters, 179)))

In [ ]:
def determine_corrected_diff(predicted_parameters, ground_truth_parameters):
    flipped_predicted_parameters = flip_parameters(predicted_parameters)
    
    non_flipped_diff = determine_diff_with_rotation(predicted_parameters, ground_truth_parameters)
    flipped_diff = determine_diff_with_rotation(flipped_predicted_parameters, ground_truth_parameters)
    
    non_flipped_sum_sqrd = tf.reduce_sum(
        non_flipped_diff**2, axis=-1
    )
    flipped_sum_sqrd = tf.reduce_sum(
        flipped_diff**2, axis=-1
    )
    
    minimisation_stack = tf.stack([
        non_flipped_sum_sqrd, flipped_sum_sqrd
    ])
#     print(minimisation_stack)
    
    min_index = tf.argmin(minimisation_stack, axis=0)
    
#     print(min_index)
    
    both_diffs = tf.stack([
        non_flipped_diff, flipped_diff
    ])
#     print(both_diffs.numpy()[min_index, :, :])

    diff = tf.gather(
        both_diffs, min_index, axis=0
    )
    
#     diff = tf.convert_to_tensor(both_diffs.numpy()[min_index, :, :])
    return diff


for parameters, img in create_pipeline_dataset(4).take(1):
    flipped_parameters = flip_parameters(parameters)
    print(determine_corrected_diff(parameters, parameters))
    print(determine_corrected_diff(parameters, flipped_parameters))
    print(determine_corrected_diff(flipped_parameters, parameters))

In [ ]:
def loss_function(predicted_parameters, ground_truth_parameters):
    diff = determine_corrected_diff(predicted_parameters, ground_truth_parameters)
    loss = diff ** 2

    return loss


for parameters, img in create_pipeline_dataset(2).take(1):
    print(parameters)
    print(loss_function(parameters, parameters))
    print(loss_function(parameters, parameters + 1))
    print(loss_function(parameters, parameters + 0.5))

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model)

In [ ]:
BATCH_SIZE = 10

test_dataset = create_pipeline_dataset(1)
train_dataset = create_pipeline_dataset(BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1, beta_1=0.5)


@tf.function(experimental_relax_shapes=True)
def train_step(ground_truth_parameters, input_image):
    with tf.GradientTape() as tape:
        predicted_parameters = model(input_image, training=True)
        loss = loss_function(predicted_parameters, ground_truth_parameters)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    reduced_loss = tf.reduce_mean(loss)
    return reduced_loss
        
        
concrete_train_step = train_step.get_concrete_function(
    tf.TensorSpec(shape=[BATCH_SIZE, 8], dtype=tf.float32), 
    tf.TensorSpec(shape=[BATCH_SIZE, None, None], dtype=tf.float32)
)

In [ ]:
def fit(train_ds, epochs, test_ds):
    logging_index = tf.convert_to_tensor(0, dtype=tf.int64)
    
    for epoch in range(epochs):
        start = time.time()

        for parameters, img in test_ds.take(1):
            results_figures(model, img, parameters)

        iters_per_epoch = 10
        for parameters, img in tqdm.tqdm(train_ds.take(iters_per_epoch), total=iters_per_epoch):
            reduced_loss = concrete_train_step(parameters, img)
            
            logging_index += 1
            with summary_writer.as_default():
                tf.summary.scalar('loss', reduced_loss, step=logging_index)
            
        
        if (epoch + 1) % 20 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                            time.time()-start))
    checkpoint.save(file_prefix = checkpoint_prefix)

```bash
poetry run tensorboard --logdir examples/site-specific/cancer-care-associates/production/Winston\ Lutz/prototyping/tf_model/logs/
```

In [ ]:
EPOCHS = 150

log_dir="logs/"
summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

fit(train_dataset, EPOCHS, test_dataset)